#### ID 2100

```Given the education levels and salaries of a group of individuals, find what is the average salary for each level of education.```

In [ ]:
%%sql
SELECT education, AVG(salary) AS avg_salary
FROM google_salaries
GROUP BY education

In [ ]:
df = google_salaries

df.groupby('education', as_index=False).agg(avg_salary=('salary', 'mean'))

#### ID 2107

```Write a query to return all Customers (cust_id) who are violating primary key constraints in the Customer Dimension (dim_customer) i.e. those Customers who are present more than once in the Customer Dimension. For example if cust_id 'C123' is present thrice then the query should return 'C123' | '3' as output.```

In [ ]:
%%sql
SELECT cust_id, count(cust_id) AS n_occurences
FROM dim_customer
GROUP BY cust_id
HAVING count(cust_id) >= 2

In [ ]:
df = dim_customer

df.groupby('cust_id', as_index=False).agg(n_occurences=('cust_id', 'count')).query('n_occurences >= 2')

#### ID 2108

```Each Employee is assigned one territory and is responsible for the Customers from this territory. There may be multiple employees assigned to the same territory. Write a query to get the Employees who are responsible for the maximum number of Customers. Output the Employee ID and the number of Customers.```

In [ ]:
%%sql
WITH cte AS (SELECT empl_id, COUNT(*) OVER (PARTITION BY empl_id) AS n_customers
             FROM map_employee_territory et
                      JOIN map_customer_territory ct USING (territory_id)
             ORDER BY empl_id)
SELECT DISTINCT empl_id, n_customers
FROM cte
WHERE n_customers = (SELECT MAX(n_customers) FROM cte)

In [ ]:
df = pd.merge(map_customer_territory, map_employee_territory, how='inner', on='territory_id')

df_grouped = df.groupby('empl_id', as_index=False).agg(n_customers=('cust_id', 'count'))

max_n_customers = df_grouped.nlargest(1, 'n_customers')['n_customers'].to_list()

df_grouped.query('n_customers == @max_n_customers')

#### ID 2109

```Write a query to get a list of products that have not had any sales. Output the ID and market name of these products.```

In [ ]:
%%sql
SELECT dp.prod_sku_id, dp.market_name
FROM dim_product AS dp
         LEFT JOIN fct_customer_sales AS cs ON dp.prod_sku_id = cs.prod_sku_id
WHERE cs.prod_sku_id IS NULL

In [ ]:
df = pd.merge(dim_product, fct_customer_sales, how='left', left_on='prod_sku_id', right_on='prod_sku_id').query(
    'cust_id.isnull()')[['prod_sku_id', 'market_name']]